### Import

In [ ]:
#import potrzebnych bibliotek
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm

### Wczytanie obu instancji

In [ ]:
#wczytanie pierwszej instancji (kroA200)

instance = pd.read_csv('kroA200.tsp', sep=' ', names=['node', 'x', 'y'], skiprows=6, skipfooter=1,
                      engine='python')
kroA200 = []

for index1, row1 in instance.iterrows():
    tmp = []
    for index2, row2 in instance.iterrows():
        if (index1==index2):
            tmp.append(0)
        else:
            tmp.append(distance(row1['x'],row2['x'],row1['y'],row2['y']))
    kroA200.append(tmp)

np.array(kroA200)

In [ ]:
#wczytanie drugiej instancji (kroB200)

instance2 = pd.read_csv('kroB200.tsp', sep=' ', names=['node', 'x', 'y'], skiprows=6, skipfooter=1,
                      engine='python')

kroB200 = []

for index1, row1 in instance2.iterrows():
    tmp = []
    for index2, row2 in instance2.iterrows():
        if (index1==index2):
            tmp.append(0)
        else:
            tmp.append(distance(row1['x'],row2['x'],row1['y'],row2['y']))
    kroB200.append(tmp)

np.array(kroB200)

### Funkcje pomocnicze - tworzenie cyklu

In [ ]:
def get_start_nodes(matrix):
    first_start=rd.randint(0, 199)
    second_start=np.argmax(matrix[first_start])
    return first_start,second_start

In [ ]:
def create_cycles(first_start, second_start):
    cycle1=[]
    cycle2=[]
    remaining = list(range(200))
    remaining.remove(first_start)
    remaining.remove(second_start)
    cycle1.append(first_start)
    cycle2.append(second_start)
    while remaining:
        node = rd.choice(remaining)
        cycle1.append(node)
        remaining.remove(node)
        node = rd.choice(remaining)
        cycle2.append(node)
        remaining.remove(node)
    return cycle1, cycle2

In [ ]:
def create_cycles_for_weight(first_start, second_start):
    cycle1=[]
    cycle2=[]
    remaining = list(range(200))
    remaining.remove(first_start)
    remaining.remove(second_start)
    cycle1.append(first_start)
    cycle2.append(second_start)
    return cycle1, cycle2, remaining

Utworzenie losowego cyklu

In [ ]:
def create_random_cycles(instance):
    vertices = np.arange(0, instance.shape[0])
    rd.shuffle(vertices)
    
    split_index = instance.shape[0] // 2
    cycle1, cycle2 = vertices[:split_index], vertices[split_index:]
    
    return cycle1, cycle2

### Funkcje pomocnicze - długość cyklu

In [ ]:
#funkcja obliczająca odległość między dwoma wierzchołkami z matematycznym zaokrągleniem
def distance(x1,x2,y1,y2):
    return np.floor(np.sqrt((x1-x2)**2+(y1-y2)**2)+0.5)

In [ ]:
#funkcja obliczająca całkowitą długość cyklu
def distance_sum(cycle,matrix):
    distance = 0
    for i in range(len(cycle)):
        if i==len(cycle)-1:
            distance+=matrix[cycle[i]][cycle[0]]
        else:
            distance+=matrix[cycle[i]][cycle[i+1]]
    return distance

### Funkcje pomocnicze - obliczenie delty funkcji celu

In [ ]:
#funkcja obliczająca różnicę w długości przy wymianie wierzchołków między dwoma cyklami
def difference_change_nodes_between_cycles(matrix, cycle1, cycle2, node1, node2):
    index1 = cycle1.index(node1)
    index2 = cycle2.index(node2)
    old_distance = matrix[node1][cycle1[(index1-1)%len(cycle1)]]+matrix[node1][cycle1[(index1+1)%len(cycle1)]]+matrix[node2][cycle2[(index2-1)%len(cycle2)]]+matrix[node2][cycle2[(index2+1)%len(cycle2)]]
    new_distance = matrix[node1][cycle2[(index2-1)%len(cycle2)]]+matrix[node1][cycle2[(index2+1)%len(cycle2)]]+matrix[node2][cycle1[(index1-1)%len(cycle1)]]+matrix[node2][cycle1[(index1+1)%len(cycle1)]]
    return new_distance - old_distance

In [ ]:
#funkcja obliczająca różnicę w długości przy wymianie krawędzi w tym samym cyklu
def difference_change_edges_in_cycle(matrix, cycle1, node1, node2):
    index1 = cycle1.index(node1)
    index2 = cycle1.index(node2)
    old_distance = matrix[node1][cycle1[(index1+1)%len(cycle1)]]+matrix[node2][cycle1[(index2+1)%len(cycle1)]]
    new_distance = matrix[node1][node2]+matrix[cycle1[(index1+1)%len(cycle1)]][cycle1[(index2+1)%len(cycle1)]]
    return new_distance - old_distance

Obliczenie delty dla listy ruchów przynoszących poprawę

In [ ]:
def calculate_delta(cycle1, cycle2, matrix, move):
    # move between cycles
    if not move[0]:
        index1, index2 = move[1][0], move[1][1]
        
        node1, node2 = cycle1[index1], cycle2[index2]
        node1_before, node2_before = cycle1[index1-1], cycle2[index2-1]
        node1_after, node2_after = cycle1[(index1+1) % len(cycle1)], cycle2[(index2+1) % len(cycle2)]
        
        delta = matrix[node1_before][node2] + matrix[node2][node1_after] + \
                matrix[node2_before][node1] + matrix[node1][node2_after] - \
                matrix[node1_before][node1] - matrix[node1][node1_after] - \
                matrix[node2_before][node2] - matrix[node2][node2_after]
     
    # move in cycle
    else:
        cycle = cycle1 if not move[1][0] else cycle2
        index1, index1_2, index2, index2_2 = move[1][1:]
        node1, node1_2, node2, node2_2 = cycle[index1], cycle[index1_2], cycle[index2], cycle[index2_2]
        
        delta = matrix[node1][node2] + matrix[node1_2][node2_2] - matrix[node1][node1_2] - matrix[node2][node2_2]
        
    return delta

### Funkcje pomocnicze - ruchy kandydackie

In [ ]:
#funkcja zwracająca 10 najbliższych sąsiadów
def ten_neighbours (matrix, node):
    nodes_to_check = matrix[node]
    sorted_nodes_to_check = sorted(nodes_to_check)
    result = []
    for value in sorted_nodes_to_check[1:11]:
        result.append(nodes_to_check.index(value))
    return result

In [ ]:
def check_moves(matrix, cycle1, cycle2):
    # 1 - cycle1 lub edge
    # 2 - cycle2 lub node
    all_moves = []
    for i in range (len(matrix)):
        neighbours = ten_neighbours(matrix, i)
        for neighbour in neighbours:
            best_difference = 0
            if (i in cycle1 and neighbour in cycle1):
                difference1 = difference_change_edges_in_cycle(matrix, cycle1, i, neighbour)
                difference2 = difference_change_edges_in_cycle(matrix, cycle1, cycle1[(cycle1.index(i)-1)%len(cycle1)], cycle1[(cycle1.index(neighbour)-1)%len(cycle1)])
                best_difference = min([difference1, difference2])
                if best_difference < 0:
                    if best_difference == difference1:
                        all_moves.append([difference1, i, neighbour, 1, 1])
                    else:
                        all_moves.append([difference2, cycle1[(cycle1.index(i)-1)%len(cycle1)], cycle1[(cycle1.index(neighbour)-1)%len(cycle1)], 1, 1])
            elif (i in cycle2 and neighbour in cycle2):
                difference1 = difference_change_edges_in_cycle(matrix, cycle2, i, neighbour)
                difference2 = difference_change_edges_in_cycle(matrix, cycle2, cycle2[(cycle2.index(i)-1)%len(cycle2)], cycle2[(cycle2.index(neighbour)-1)%len(cycle2)])
                best_difference = min([difference1, difference2])
                if best_difference < 0:
                    if best_difference == difference1:
                        all_moves.append([difference1, i, neighbour, 2, 1])
                    else:
                        all_moves.append([difference2, cycle2[(cycle2.index(i)-1)%len(cycle2)], cycle2[(cycle2.index(neighbour)-1)%len(cycle2)], 2, 1])
            elif (i in cycle1 and neighbour in cycle2):
                difference1 = difference_change_nodes_between_cycles(matrix, cycle1, cycle2, i, cycle2[(cycle2.index(neighbour)+1)%len(cycle2)])
                difference2 = difference_change_nodes_between_cycles(matrix, cycle1, cycle2, i, cycle2[(cycle2.index(neighbour)-1)%len(cycle2)])
                difference3 = difference_change_nodes_between_cycles(matrix, cycle1, cycle2, cycle1[(cycle1.index(i)+1)%len(cycle1)], neighbour)
                difference4 = difference_change_nodes_between_cycles(matrix, cycle1, cycle2, cycle1[(cycle1.index(i)-1)%len(cycle1)], neighbour)
                best_difference = min([difference1, difference2, difference3, difference4])
                if best_difference < 0:
                    if best_difference == difference1:
                        all_moves.append([difference1, i, cycle2[(cycle2.index(neighbour)+1)%len(cycle2)], 1, 2])
                    elif best_difference == difference2:
                        all_moves.append([difference2, i, cycle2[(cycle2.index(neighbour)-1)%len(cycle2)], 1, 2])
                    elif best_difference == difference3:
                        all_moves.append([difference3, cycle1[(cycle1.index(i)+1)%len(cycle1)], neighbour, 1, 2])
                    else:
                        all_moves.append([difference4, cycle1[(cycle1.index(i)-1)%len(cycle1)], neighbour, 1, 2])
            else:
                difference1 = difference_change_nodes_between_cycles(matrix, cycle2, cycle1, i, cycle1[(cycle1.index(neighbour)+1)%len(cycle1)])
                difference2 = difference_change_nodes_between_cycles(matrix, cycle2, cycle1, i, cycle1[(cycle1.index(neighbour)-1)%len(cycle1)])
                difference3 = difference_change_nodes_between_cycles(matrix, cycle2, cycle1, cycle2[(cycle2.index(i)+1)%len(cycle2)], neighbour)
                difference4 = difference_change_nodes_between_cycles(matrix, cycle2, cycle1, cycle2[(cycle2.index(i)-1)%len(cycle2)], neighbour)
                best_difference = min([difference1, difference2, difference3, difference4])
                if best_difference < 0:
                    if best_difference == difference1:
                        all_moves.append([difference1, i, cycle1[(cycle1.index(neighbour)+1)%len(cycle1)], 2, 2])
                    elif best_difference == difference2:
                        all_moves.append([difference2, i, cycle1[(cycle1.index(neighbour)-1)%len(cycle1)], 2, 2])
                    elif best_difference == difference3:
                        all_moves.append([difference3, cycle2[(cycle2.index(i)+1)%len(cycle2)], neighbour, 2, 2])
                    else:
                        all_moves.append([difference4, cycle2[(cycle2.index(i)-1)%len(cycle2)], neighbour, 2, 2])
    all_moves=np.array(all_moves, dtype = int)
    if len(all_moves) == 0:
        return []
    return all_moves[all_moves[:, 0].argsort()]

In [ ]:
#funkcja zwracająca listę wierzchołków, dla których najlepsze miejsce wstawienia to miejsce w cyklu o podanym indeksie
def nodes_with_best_given_index(matrix,cycle,remaining,given_index):
    result_nodes = []
    for node_to_add in remaining:
        best1 = 9999999999998
        best2 = 9999999999999
        index = -1
        for i in range (len(cycle)):
            if i!=len(cycle)-1:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[i+1]][node_to_add] - matrix[cycle[i]][cycle[i+1]]
            else:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[0]][node_to_add] - matrix[cycle[i]][cycle[0]]
            if tmp_distance < best2:
                if tmp_distance < best1:
                    best2 = best1
                    best1 = tmp_distance
                    index = i
                else:
                    best2 = tmp_distance
        if index == given_index:
            result_nodes.append([node_to_add,best1,best2])
    return result_nodes

In [ ]:
def greedy_regret_with_weight(matrix, cycle, remaining):
    best_node_to_add, best_index = greedy_cycle(matrix, cycle, remaining)
    nodes_on_the_same_position = nodes_with_best_given_index(matrix, cycle, remaining, best_index)
    return calculate_regret_with_weight(nodes_on_the_same_position), best_index

In [ ]:
def calculate_regret_with_weight(list_nodes):
    list_nodes = np.array(list_nodes)
    difference = list_nodes[:,2]-1.2*(list_nodes[:,1])
    best_result_index = np.argmax(difference)
    return int(list_nodes[best_result_index][0])

In [ ]:
#algorytm zachłanny inspirowany metodą rozbudowy cyklu
def greedy_cycle(matrix, cycle, remaining):
    min_distance = 999999999
    index = -1
    min_node = cycle[0]
    for node_to_add in remaining:
        for i in range (len(cycle)):
            if i!=len(cycle)-1:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[i+1]][node_to_add] - matrix[cycle[i]][cycle[i+1]]
            else:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[0]][node_to_add] - matrix[cycle[i]][cycle[0]]
            if tmp_distance < min_distance:
                min_distance = tmp_distance
                min_node = node_to_add
                index = i
    return min_node, index

In [ ]:
#algorytm zachłanny inspirowany metodą najbliższego sąsiada
def nearest_neighbour(matrix, cycle, remaining):
    min_distance = matrix[cycle[0]][remaining[0]]
    min_node = remaining[0]
    node_cycle = cycle[0]
    for node_to_add in remaining:
        for node_in_cycle in cycle:
            if matrix[node_to_add][node_in_cycle] < min_distance:
                min_distance = matrix[node_to_add][node_in_cycle]
                min_node = node_to_add
                node_cycle = node_in_cycle
    return min_node, node_cycle

In [ ]:
def calculate_greedy_regret_with_weight(matrix, cycle1, cycle2, remaining):
    node_to_add, node_in_cycle = nearest_neighbour(matrix, cycle1, remaining)
    index = cycle1.index(node_in_cycle)
    cycle1.insert(index+1, node_to_add)
    remaining.remove(node_to_add)

    node_to_add, node_in_cycle = nearest_neighbour(matrix, cycle2, remaining)
    index = cycle2.index(node_in_cycle)
    cycle2.insert(index+1, node_to_add)
    remaining.remove(node_to_add)

    node_to_add, index = greedy_cycle(matrix, cycle1, remaining)
    cycle1.insert(index+1, node_to_add)
    remaining.remove(node_to_add)

    node_to_add, index = greedy_cycle(matrix, cycle2, remaining)
    cycle2.insert(index+1, node_to_add)
    remaining.remove(node_to_add)

    while (remaining):
        node_to_add, index = greedy_regret_with_weight(matrix, cycle1, remaining)
        index = int(index)
        cycle1.insert(index+1, node_to_add)
        remaining.remove(node_to_add)
        if not remaining:
             break
        node_to_add, index = greedy_regret_with_weight(matrix, cycle2, remaining)
        index = int(index)
        cycle2.insert(index+1, node_to_add)
        remaining.remove(node_to_add)
    return([distance_sum(cycle1,matrix)+distance_sum(cycle2,matrix), cycle1, cycle2])

In [ ]:
def steepest_change_edges(cycle, matrix):
    best_difference = None
    best_node1 = None
    best_node2 = None
    possible_pairs = []
    for i in range(len(cycle)-2):
        for j in range(i+2,len(cycle)):
            possible_pairs.append([i,j])
    shuffled_pairs=possible_pairs
    rd.shuffle(shuffled_pairs)
    for pair in shuffled_pairs:
        difference = difference_in_edges(matrix,cycle,pair[0],pair[1])
        if best_difference is None or difference < best_difference:
            best_difference = difference
            best_node1 = pair[0]
            best_node2 = pair[1]
    if best_difference>=0:
        return None
    cycle_copy=cycle
    result = cycle_copy[:best_node1]+list(reversed(cycle_copy[best_node1:best_node2+1]))+cycle_copy[best_node2+1:]
    return result

In [ ]:
def difference_in_new_cycles(matrix,cycle1,cycle2,node1,node2):
    
    ind1=cycle1.index(node1)
    ind2=cycle2.index(node2)
    
    cycle1_len = len(cycle1)
    cycle2_len = len(cycle2)
    
    node_before_1, node_before_2 = cycle1[(ind1-1) % cycle1_len], cycle2[(ind2-1) % cycle2_len]
    node_after_1, node_after_2 = cycle1[(ind1+1) % cycle1_len], cycle2[(ind2+1) % cycle2_len]
    
    old_distance = matrix[node_before_1][node1] + matrix[node1][node_after_1] + \
                matrix[node_before_2][node2] + matrix[node2][node_after_2] 
    
    new_distance = matrix[node_before_1][node2] + matrix[node2][node_after_1] + \
                matrix[node_before_2][node1] + matrix[node1][node_after_2]
    
    return new_distance-old_distance

In [ ]:
def steepest_change_nodes(cycle1, cycle2, matrix):
    best_difference = None
    best_node1 = None
    best_node2 = None
    shuffled_cycle1=cycle1
    shuffled_cycle2=cycle2
    rd.shuffle(shuffled_cycle1)
    rd.shuffle(shuffled_cycle2)
    for node1 in shuffled_cycle1:
        for node2 in shuffled_cycle2:
            difference = difference_in_new_cycles(matrix,cycle1,cycle2,node1,node2)
            if best_difference is None or difference < best_difference:
                best_difference = difference
                best_node1 = node1
                best_node2 = node2
    if best_difference>=0:
        return None, None
    cycle1_copy=cycle1
    cycle2_copy=cycle2
    ind1=cycle1_copy.index(best_node1)
    ind2=cycle2_copy.index(best_node2)
    cycle1_copy[ind1]=best_node2
    cycle2_copy[ind2]=best_node1
    return cycle1_copy, cycle2_copy

In [ ]:
def difference_in_edges(matrix,cycle,ind1,ind2):
    cycle_len = len(cycle)
    
    node_after_1, node_after_2 = cycle[(ind1+1) % cycle_len], cycle[(ind2+1) % cycle_len]    
    
    old_distance = matrix[ind1][node_after_1] + matrix[ind2][node_after_2]
    
    new_distance = matrix[ind1][ind2] +  matrix[node_after_1][node_after_2]
    
    return new_distance-old_distance

In [ ]:
def measure_steepest_edges(matrix,cycle1,cycle2):
    start = time.time()
    changes = [0, 1, 2]
    result_cycle1 = cycle1
    result_cycle2 = cycle2
    while True:
        changes = [0, 1, 2]
        result_cycle1_c1 = None
        result_cycle2_c1 = None
        result_cycle1_c2 = None
        result_cycle2_c3 = None
        c1 = None
        c2 = None
        c3 = None
        for i in reversed(range(len(changes))):
            if changes[i] == 0:
                candidate1, candidate2 = steepest_change_nodes(result_cycle1,result_cycle2,matrix)
                if candidate1 is not None:
                    result_cycle1_c1 = candidate1
                    result_cycle2_c1 = candidate2
                    c1 = (distance_sum(result_cycle1_c1, matrix)+distance_sum(result_cycle2_c1, matrix))-(distance_sum(cycle1, matrix)+distance_sum(cycle2, matrix))
            elif changes[i] == 1:
                candidate1 = steepest_change_edges(result_cycle1, matrix)
                if candidate1 is not None:
                    result_cycle1_c2 = candidate1
                    c2 = distance_sum(result_cycle1_c2, matrix) - distance_sum(cycle1, matrix)
            else:
                candidate2 = steepest_change_edges(result_cycle2, matrix)
                if candidate2 is not None:
                    result_cycle2_c3 = candidate2
                    c3 = distance_sum(result_cycle2_c3, matrix) - distance_sum(cycle2, matrix)
        c_list = [c1, c2, c3]
        if all(c is None for c in c_list):
            break
        best = min(c for c in c_list if c is not None)
        if best == c1:
            result_cycle1=result_cycle1_c1
            result_cycle2=result_cycle2_c1
        elif best == c2:
            result_cycle1 = result_cycle1_c2
        else:
            result_cycle2 = result_cycle2_c3
    total_distance = distance_sum(result_cycle1, matrix)+distance_sum(result_cycle2, matrix)
    return [total_distance, result_cycle1, result_cycle2]

### Funkcje pomocnicze - lista ruchów przynoszących poprawę

Wygenerowanie wszystkich możliwych ruchów

In [ ]:
def all_possible_moves(cycle1, cycle2):
    # create list of tuples (move_type, (indexes in cycles), (node_before, current_node, node_after) x2)
    # move_type = 0 => move between cycles
    # move_type = 1 => move in cycle
    all_moves = []
    
    # possible moves between cycles
    for index1 in range(len(cycle1)):
        for index2 in range(len(cycle2)):
            move = (0, (index1, index2), 
                   (cycle1[(index1-1) % len(cycle1)], cycle1[index1], cycle1[(index1+1) % len(cycle1)]),
                   (cycle2[(index2-1) % len(cycle2)], cycle2[index2], cycle2[(index2+1) % len(cycle2)]))
            
            all_moves.append(move)
            
    # possible moves in cycle - edge replacement
    for cycle_number, cycle in enumerate([cycle1, cycle2]):
        for index1 in range(len(cycle)-1):
            for index2 in range(len(cycle)-1):
                if abs(index1-index2) < 2:
                    continue
                    
                if index1 == len(cycle)-1:
                    index1_2 = 0
                else:
                    index1_2 = index1+1
                    
                if index2 == len(cycle)-1:
                    index2_2 = 0
                else:
                    index2_2 = index2+1
                    
                move = (1, (cycle_number, index1, index1_2, index2, index2_2), 
                        (cycle[index1], cycle[index1_2], cycle[index2], cycle[index2_2]))
                
                all_moves.append(move)
    
    return all_moves

Wykonanie wybranego ruchu

In [ ]:
def apply_move(cycle1, cycle2, move):
    # if move is beetwen cycles
    if not move[0]:
        temp = cycle1[move[1][0]]
        cycle1[move[1][0]] = cycle2[move[1][1]]
        cycle2[move[1][1]] = temp
     
    # if move is in cycle
    else:
        cycle = cycle1 if not move[1][0] else cycle2
        index1, index1_2, index2, index2_2 = move[1][1:]
        if index1_2 == 0:
            c1 = [cycle[index1_2]]
            c2 = cycle[index2_2:index1 + 1]
            c3 = cycle[index2:index1_2:-1]
        elif index2_2 == 0:
            c1 = cycle[:index1_2]
            c2 = cycle[index2:index1:-1]
            c3 = []
        else:
            c1 = cycle[:min(index1_2, index2_2)]
            c2 = cycle[max(index1_2, index2_2)-1 : min(index1_2, index2_2)-1 : -1]
            c3 = cycle[max(index1_2, index2_2):]
#         print('c1: ', c1)
#         print('c2: ', c2)
#         print('c3: ', c3)
        
        if type(c1) is np.ndarray:
            c1 = c1.tolist()
        if type(c2) is np.ndarray:
            c2 = c2.tolist()
        if type(c3) is np.ndarray:
            c3 = c3.tolist()
            
        if not isinstance(c1, list):
            c1 = [c1]
        if not isinstance(c2, list):
            c2 = [c2]
        if not isinstance(c3, list):
            c3 = [c3]
        
        cycle = c1 + c2 + c3
        
        if len(cycle) != len(cycle1):
#             print('l1' , len(c1))
#             print('l2' , len(c2))
#             print('l3' , len(c3))
            print(cycle)
            print("APPLY: ", len(cycle))
            print('c1', cycle1)
            print('c2', cycle2)
            print("\n")
            print(index1, index1_2, index2, index2_2)
        
        if not move[1][0]:
            cycle1 = cycle
        else:
            cycle2 = cycle
            
    return cycle1, cycle2

Wygenerowanie nowych możliwych ruchów między cyklami

In [ ]:
def new_moves_between_cycles(cycle1, cycle2, index1, index2):
    new_moves = []
    
    if index1 is None:
        for j in ((index2-1) % len(cycle2), index2, (index2+1) % len(cycle2)):
            for i in range(len(cycle1)):
                move = (0, (i, j),
                        (cycle1[(i-1) % len(cycle1)], cycle1[i], cycle1[(i+1) % len(cycle1)]),
                        (cycle2[(j-1) % len(cycle2)], cycle2[j], cycle2[(j+1) % len(cycle2)]))
                new_moves.append(move)

    elif index2 is None:
        for i in ((index1-1) % len(cycle1), index1, (index1+1) % len(cycle1)):
            for j in range(len(cycle2)):
                move = (0, (i, j),
                        (cycle1[(i-1) % len(cycle1)], cycle1[i], cycle1[(i+1) % len(cycle1)]),
                        (cycle2[(j-1) % len(cycle2)], cycle2[j], cycle2[(j+1) % len(cycle2)]))
                new_moves.append(move)
                
    else:
        for i in ((index1-1) % len(cycle1), index1, (index1+1) % len(cycle1)):
            for j in range(len(cycle2)):
                move = (0, (i, j),
                        (cycle1[(i-1) % len(cycle1)], cycle1[i], cycle1[(i+1) % len(cycle1)]),
                        (cycle2[(j-1) % len(cycle2)], cycle2[j], cycle2[(j+1) % len(cycle2)]))
                new_moves.append(move)
                
        for j in ((index2-1) % len(cycle2), index2, (index2+1) % len(cycle2)):
            for i in range(len(cycle1)):
                move = (0, (i, j),
                        (cycle1[(i-1) % len(cycle1)], cycle1[i], cycle1[(i+1) % len(cycle1)]),
                        (cycle2[(j-1) % len(cycle2)], cycle2[j], cycle2[(j+1) % len(cycle2)]))
                new_moves.append(move)
                
    return new_moves        

Wygenerowanie nowych możliwych ruchów w cyklu

In [ ]:
def new_moves_in_cycle(cycle_number, cycle, index1):
    new_moves = []
    
#     print(index1)
    
    for index2 in range(len(cycle)-1):
        if abs(index1-index2) < 2:
            continue
            
        if index1 == len(cycle)-1:
            index1_2 = 0
        else:
            index1_2 = index1+1
            
        if index2 == len(cycle)-1:
            index2_2 = 0
        else:
            index2_2 = index2+1        
            
#         print("LEN: ", len(cycle))
#         print("I1: ", index1)
#         print("I1_2: ", index1_2)
#         print("I2: ", index2)
#         print("I2_2: ", index2_2)
        
        move = (1, (cycle_number, index1, index1_2, index2, index2_2), 
                (cycle[index1], cycle[index1_2], cycle[index2], cycle[index2_2]))
        new_moves.append(move)
        
    return new_moves

<b> Główna funkcja rozwiązująca problem lokalnego przeszukiwania z listą ruchów przynoszących poprawę

In [ ]:
def steepest_with_move_list(cycle1, cycle2, matrix):
    start = time.time()
    cycle1_copy, cycle2_copy = cycle1.copy(), cycle2.copy()
    LM = []
    
    # Remove moves for which delta >= 0
    moves = all_possible_moves(cycle1_copy, cycle2_copy)
    delta_list = []
    for move in moves:
        delta = calculate_delta(cycle1_copy, cycle2_copy, matrix, move)
        if delta < 0:
            LM.append(move)
            delta_list.append(delta)
            
    LM = (np.array(LM, dtype=object)[np.argsort(delta_list)]).tolist()
    delta_list.sort()
#     delta_list.clear()
    
    i = 0
    max_i = 99999999
    moves_to_add = []
    # Z tym i to tak w razie czego xddd
    while i < max_i:
        # check if move in list moves_to_add has delta < 0
        for move in moves_to_add:
            delta = calculate_delta(cycle1_copy, cycle2_copy, matrix, move)
            if delta < 0:
                LM.append(move)
                delta_list.append(delta)
              
        # sort LM by delta_list, sort delta_list
        LM = (np.array(LM, dtype=object)[np.argsort(delta_list)]).tolist()
        delta_list.sort()
        moves_to_add.clear()
        
        for move in LM:
#             print("L1: ", len(cycle1), "L2: ", len(cycle2))
            # exchange vertices between cycles
            if not move[0]:
                cycle1_edges = [move[2][0], move[2][2]]
                cycle2_edges = [move[3][0], move[3][2]]
                index1, index2 = move[1][0], move[1][1]
                
                if (cycle1_copy[(index1-1)%len(cycle1)] in cycle1_edges and
                    cycle1_copy[(index1)%len(cycle1)] == move[2][1] and
                    cycle1_copy[(index1+1)%len(cycle1)] in cycle1_edges and
                    cycle2_copy[(index2-1)%len(cycle2)] in cycle2_edges and
                    cycle2_copy[(index2)%len(cycle2)] == move[3][1] and
                    cycle2_copy[(index2+1)%len(cycle2)] in cycle2_edges):
                    
#                     delta = calculate_delta(cycle1_copy, cycle2_copy, matrix, move)
                    cycle1_copy, cycle2_copy = apply_move(cycle1_copy, cycle2_copy, move)
                    
                    # add moves to list moves_to_add
                    moves_to_add.extend(new_moves_between_cycles(cycle1_copy, cycle2_copy, move[1][0], move[1][1]))
                    moves_to_add.extend(new_moves_in_cycle(0, cycle1_copy, (move[1][0]-1) % len(cycle1)))
                    moves_to_add.extend(new_moves_in_cycle(0, cycle1_copy, move[1][0]))
                    
                    moves_to_add.extend(new_moves_in_cycle(1, cycle2_copy, (move[1][1]-1) % len(cycle2)))
                    moves_to_add.extend(new_moves_in_cycle(1, cycle2_copy, move[1][1]))
                    
                    index = LM.index(move)
                    # remove from delta list
                    del delta_list[index]
                    LM.remove(move)
                    break
                    
                else:
                    index = LM.index(move)
                    # remove from delta list
                    del delta_list[index]
                    LM.remove(move)
                    
            else:
                cycle = cycle1_copy if not move[1][0] else cycle2_copy
                if cycle[move[1][1]] == move[2][0] and cycle[move[1][2]] == move[2][1] and \
                        cycle[move[1][3]] == move[2][2] and cycle[move[1][4]] == move[2][3]:
                    
#                     delta = calculate_delta(cycle1_copy, cycle2_copy, matrix, move)
                    cycle1_copy, cycle2_copy = apply_move(cycle1_copy, cycle2_copy, move)

                    # new moves
                    cycle = cycle1_copy if not move[1][0] else cycle2_copy
                    moves_to_add.extend(new_moves_in_cycle(move[1][0], cycle, move[1][1]))
                    moves_to_add.extend(new_moves_in_cycle(move[1][0], cycle, move[1][3]))
                    
                    if not move[1][0]:
                        moves_to_add.extend(new_moves_between_cycles(cycle1_copy, cycle2_copy, move[1][1], None))
                        moves_to_add.extend(new_moves_between_cycles(cycle1_copy, cycle2_copy, move[1][2], None))
                        moves_to_add.extend(new_moves_between_cycles(cycle1_copy, cycle2_copy, move[1][3], None))
                        moves_to_add.extend(new_moves_between_cycles(cycle1_copy, cycle2_copy, move[1][4], None))
                        
                    else:
                        moves_to_add.extend(new_moves_between_cycles(cycle1_copy, cycle2_copy, None, move[1][1]))
                        moves_to_add.extend(new_moves_between_cycles(cycle1_copy, cycle2_copy, None, move[1][2]))
                        moves_to_add.extend(new_moves_between_cycles(cycle1_copy, cycle2_copy, None, move[1][3]))
                        moves_to_add.extend(new_moves_between_cycles(cycle1_copy, cycle2_copy, None, move[1][4]))
                    
                    index = LM.index(move)
                    # remove from delta list
                    del delta_list[index]
                    LM.remove(move)
                    break
                    
                else:
                    index = LM.index(move)
                    # remove from delta list
                    del delta_list[index]
                    LM.remove(move)
         
        i = i + 1
        if len(LM) == 0:
            break
            
    return [distance_sum(cycle1_copy, matrix)+distance_sum(cycle2_copy, matrix), cycle1_copy, cycle2_copy]

### Testy

In [ ]:
def tests (matrix, cycle1,cycle2):
    moves = check_moves(matrix, cycle1, cycle2)
    while len(moves)>0:
        best_move = moves[0]
        if best_move[4] == 1:
            if best_move[3] == 1:
                index1 = (cycle1.index(best_move[1])+1)%len(cycle1)
                index2 = (cycle1.index(best_move[2])+1)%len(cycle1)
                if index1<index2:
                    cycle1 = cycle1[:index1]+list(reversed(cycle1[index1:index2]))+cycle1[index2:]
                else:
                    cycle1 = list(reversed(cycle1[:index2]))+list(reversed(cycle1[index1:]))+cycle1[index2:index1]
            else:
                index1 = (cycle2.index(best_move[1])+1)%len(cycle2)
                index2 = (cycle2.index(best_move[2])+1)%len(cycle2)
                if index1<index2:
                    cycle2 = cycle2[:index1]+list(reversed(cycle2[index1:index2]))+cycle2[index2:]
                else:
                    cycle2 = list(reversed(cycle2[:index2]))+list(reversed(cycle2[index1:]))+cycle2[index2:index1]
        else:
            if best_move[3] == 1:
                index1 = cycle1.index(best_move[1])
                index2 = cycle2.index(best_move[2])
                cycle1[index1] = best_move[2]
                cycle2[index2] = best_move[1]
            else:
                index1 = cycle1.index(best_move[2])
                index2 = cycle2.index(best_move[1])
                cycle1[index1] = best_move[1]
                cycle2[index2] = best_move[2]
        moves = check_moves(matrix, cycle1, cycle2)
    return [distance_sum(cycle1,matrix)+distance_sum(cycle2,matrix), cycle1, cycle2]

In [ ]:
results_kroA200 = [[],[],[],[]]
results_kroB200 = [[],[],[],[]]
results_kroA200_time = [[],[],[],[]]
results_kroB200_time = [[],[],[],[]]
for i in tqdm(range(100)):
    first_start, second_start = get_start_nodes(kroA200)
    cycle1, cycle2 = create_cycles(first_start, second_start)
    copy_cycle1 = cycle1.copy()
    copy_cycle2 = cycle2.copy()
    
    start = time.time()
    results_kroA200[0].append(tests(kroA200,copy_cycle1,copy_cycle2))
    results_kroA200_time[0].append(time.time()-start)
    
    copy_cycle1 = cycle1.copy()
    copy_cycle2 = cycle2.copy()
    
    start = time.time()
    results_kroA200[1].append(steepest_with_move_list(copy_cycle1,copy_cycle2,kroA200))
    results_kroA200_time[1].append(time.time()-start)
    
    cycle1_w, cycle2_w, remaining = create_cycles_for_weight(first_start, second_start)
    start = time.time()
    results_kroA200[2].append(calculate_greedy_regret_with_weight(kroA200, cycle1_w, cycle2_w, remaining))
    results_kroA200_time[2].append(time.time()-start)
    
#     copy_cycle1 = cycle1.copy()
#     copy_cycle2 = cycle2.copy()
    
#     start = time.time()
#     results_kroA200[3].append(measure_steepest_edges(kroA200, copy_cycle1, copy_cycle2))
#     results_kroA200_time[3].append(start-time.time())
    
    
for i in tqdm(range(100)):
    first_start, second_start = get_start_nodes(kroB200)
    cycle1, cycle2 = create_cycles(first_start, second_start)
    copy_cycle1 = cycle1.copy()
    copy_cycle2 = cycle2.copy()
    
    start = time.time()
    results_kroB200[0].append(tests(kroB200,copy_cycle1,copy_cycle2))
    results_kroB200_time[0].append(time.time()-start)
    
    copy_cycle1 = cycle1.copy()
    copy_cycle2 = cycle2.copy()
    
    start = time.time()
    results_kroB200[1].append(steepest_with_move_list(copy_cycle1,copy_cycle2,kroB200))
    results_kroB200_time[1].append(time.time()-start)
    
    cycle1_w, cycle2_w, remaining = create_cycles_for_weight(first_start, second_start)
    start = time.time()
    results_kroB200[2].append(calculate_greedy_regret_with_weight(kroB200, cycle1_w, cycle2_w, remaining))
    results_kroB200_time[2].append(time.time()-start)
    
#     copy_cycle1 = cycle1.copy()
#     copy_cycle2 = cycle2.copy()
    
#     start = time.time()
#     results_kroB200[3].append(measure_steepest_edges(kroB200, copy_cycle1, copy_cycle2))
#     results_kroB200_time[3].append(start-time.time())

In [ ]:
print(results_kroA200)
print(results_kroB200)
print(results_kroA200_time)
print(results_kroB200_time)

In [ ]:
print("Ruchy kandydackie kroA200 time:")
print("Mean:", np.mean(results_kroA200_time[0]))
print("Max:", np.max(results_kroA200_time[0]))
print("Min:", np.min(results_kroA200_time[0]))
print("Lista ruchów kroA200 time:")
print("Mean:", np.mean(results_kroA200_time[1]))
print("Max:", np.max(results_kroA200_time[1]))
print("Min:", np.min(results_kroA200_time[1]))
print("Weight kro200 time:")
print("Mean:", np.mean(results_kroA200_time[2]))
print("Max:", np.max(results_kroA200_time[2]))
print("Min:", np.min(results_kroA200_time[2]))

In [ ]:
print("Ruchy kandydackie kroB200 time:")
print("Mean:", np.mean(results_kroB200_time[0]))
print("Max:", np.max(results_kroB200_time[0]))
print("Min:", np.min(results_kroB200_time[0]))
print("Lista ruchów kroB200 time:")
print("Mean:", np.mean(results_kroB200_time[1]))
print("Max:", np.max(results_kroB200_time[1]))
print("Min:", np.min(results_kroB200_time[1]))
print("Weight kroB200 time:")
print("Mean:", np.mean(results_kroB200_time[2]))
print("Max:", np.max(results_kroB200_time[2]))
print("Min:", np.min(results_kroB200_time[2]))

In [ ]:
print("Ruchy kandydackie kroA200 score:")
mean = []
for x in results_kroA200[0]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroA200[0], axis=0)[0])
print("Min:", np.min(results_kroA200[0], axis=0)[0])
print("Lista ruchów kroA200 score:")
mean = []
for x in results_kroA200[1]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroA200[1], axis=0)[0])
print("Min:", np.min(results_kroA200[1], axis=0)[0])
print("Weight kroA200 score:")
mean = []
for x in results_kroA200[2]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroA200[2], axis=0)[0])
print("Min:", np.min(results_kroA200[2], axis=0)[0])

In [ ]:
print("Ruchy kandydackie kroB200 score:")
mean = []
for x in results_kroB200[0]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroB200[0], axis=0)[0])
print("Min:", np.min(results_kroB200[0], axis=0)[0])
print("Lista ruchów kroB200 score:")
mean = []
for x in results_kroB200[1]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroB200[1], axis=0)[0])
print("Min:", np.min(results_kroB200[1], axis=0)[0])
print("Weight kroB200 score:")
mean = []
for x in results_kroB200[2]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroB200[2], axis=0)[0])
print("Min:", np.min(results_kroB200[2], axis=0)[0])

In [ ]:
plt.rc('figure', dpi=110, figsize=(9, 5))
sns.set_style('darkgrid')
    
for table in [results_kroA200[0],results_kroA200[1],results_kroA200[2]]:    
    best_result = np.argmin(table, axis=0)
    cycle1 = table[best_result[0]][1]
    cycle2 = table[best_result[0]][2]

    cycle1.append(cycle1[0])
    cycle2.append(cycle2[0])

    coordinate_x_cycle1 = []
    coordinate_y_cycle1 = []
    coordinate_x_cycle2 = []
    coordinate_y_cycle2 = []
    for node in cycle1:
        coordinate_x_cycle1.append(instance.loc[node]['x'])
        coordinate_y_cycle1.append(instance.loc[node]['y'])
    for node in cycle2:
        coordinate_x_cycle2.append(instance.loc[node]['x'])
        coordinate_y_cycle2.append(instance.loc[node]['y'])

    # plotting the line 1 points
    plt.plot(coordinate_x_cycle1, coordinate_y_cycle1, '-bo',  c='blue', mfc='k', mec='k', label="cycle1")
    plt.plot(coordinate_x_cycle2, coordinate_y_cycle2, '-bo',  c='red', mfc='k', mec='k', label="cycle2")
    # naming the x axis
    plt.xlabel('x - axis')
    # naming the y axis
    plt.ylabel('y - axis')
    # giving a title to my graph

    # show a legend on the plot
    plt.legend()

    # function to show the plot
    plt.show()

In [ ]:
plt.rc('figure', dpi=110, figsize=(9, 5))
sns.set_style('darkgrid')
    
for table in [results_kroB200[0],results_kroB200[1],results_kroB200[2]]:    
    best_result = np.argmin(table, axis=0)
    cycle1 = table[best_result[0]][1]
    cycle2 = table[best_result[0]][2]

    cycle1.append(cycle1[0])
    cycle2.append(cycle2[0])

    coordinate_x_cycle1 = []
    coordinate_y_cycle1 = []
    coordinate_x_cycle2 = []
    coordinate_y_cycle2 = []
    for node in cycle1:
        coordinate_x_cycle1.append(instance2.loc[node]['x'])
        coordinate_y_cycle1.append(instance2.loc[node]['y'])
    for node in cycle2:
        coordinate_x_cycle2.append(instance2.loc[node]['x'])
        coordinate_y_cycle2.append(instance2.loc[node]['y'])

    # plotting the line 1 points
    plt.plot(coordinate_x_cycle1, coordinate_y_cycle1, '-bo',  c='blue', mfc='k', mec='k', label="cycle1")
    plt.plot(coordinate_x_cycle2, coordinate_y_cycle2, '-bo',  c='red', mfc='k', mec='k', label="cycle2")
    # naming the x axis
    plt.xlabel('x - axis')
    # naming the y axis
    plt.ylabel('y - axis')
    # giving a title to my graph

    # show a legend on the plot
    plt.legend()

    # function to show the plot
    plt.show()